<a href="https://colab.research.google.com/github/BenjaminDKLuong/Colab_Notes/blob/master/Stock_Price_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Price Predict

In [0]:
!git clone https://github.com/PacktPublishing/Real-World-Python-Deep-Learning-Projects.git

In [0]:
# change working directory to new location
import os
os.chdir("/content/Real-World-Python-Deep-Learning-Projects/Section 5 Code/source")


In [0]:
# check current working directory
%pwd

## Data Process

## Train

## Predict